# Analysis the Reason for the Treatment Difference

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [3]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Computing Treatment Difference

In [4]:
FemaleTreatedAsMale = pd.read_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelFemalePrediction_v1.parquet'))

In [5]:
FemaleTreatedAsFemale = pd.read_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelFemalePrediction_v1.parquet'))

In [6]:
MaleTreatedAsMale = pd.read_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelMalePrediction_v1.parquet'))

In [7]:
MaleTreatedAsFemale = pd.read_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelMalePrediction_v1.parquet'))

In [8]:
TotalTreatedAsMale = pd.concat([MaleTreatedAsMale, FemaleTreatedAsMale], axis = 0)

In [9]:
TotalTreatedAsFemale = pd.concat([MaleTreatedAsFemale, FemaleTreatedAsFemale], axis = 0)

In [10]:
TotalTreatedAsMale.shape

(1911212, 3)

In [11]:
TotalTreatedAsFemale.shape

(1911212, 3)

In [12]:
TotalTreatedAsMale.head()

,index,Real_y,Predict_y
0,1322632,7.0,4.533573
1,2218620,5.0,6.462126
2,563995,2.0,3.760697
3,1467908,2.0,3.620819
4,1205253,5.0,6.468249


In [13]:
TotalTreatedAsMale = TotalTreatedAsMale.set_index('index')

In [14]:
TotalTreatedAsMale.columns = ['MaleReal_y', 'TreatedAsMale']

In [15]:
TotalTreatedAsFemale.head()

,index,Real_y,Predict_y
0,1469152,7.0,7.138159
1,708695,4.0,4.560510
2,1828455,2.0,6.806237
3,968859,8.0,5.348697
4,2375401,5.0,6.071751


In [16]:
TotalTreatedAsFemale = TotalTreatedAsFemale.set_index('index')

In [17]:
TotalTreatedAsFemale.columns = ['FemaleReal_y', 'TreatedAsFemale']

In [18]:
TotalTreatmentEffect = pd.merge(TotalTreatedAsMale, TotalTreatedAsFemale, left_index=True, right_index=True)

In [19]:
TotalTreatmentEffect.head()

,MaleReal_y,TreatedAsMale,FemaleReal_y,TreatedAsFemale
index,,,,
1322632,7.0,4.533573,7.0,4.876959
2218620,5.0,6.462126,5.0,6.798649
563995,2.0,3.760697,2.0,3.260359
1467908,2.0,3.620819,2.0,4.175891
1205253,5.0,6.468249,5.0,6.728331


In [20]:
TotalTreatmentEffect['TreatmentEffectFemMal'] = TotalTreatmentEffect['TreatedAsFemale'] - TotalTreatmentEffect['TreatedAsMale']

In [21]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [22]:
Df = pd.read_parquet(Df_Filename)

In [23]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [24]:
Df = pd.merge(TotalTreatmentEffect, Df, left_index=True, right_index=True)

In [25]:
Df.head()

,MaleReal_y,TreatedAsMale,FemaleReal_y,TreatedAsFemale,TreatmentEffectFemMal,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1322632,7.0,4.533573,7.0,4.876959,0.343386,9,11805.929318,7.0,1.0,0.0,...,1.0,0.0,0.0,73.0,5.0,6.0,1.0,2.0,3.0,HRV
2218620,5.0,6.462126,5.0,6.798649,0.336522,15,9235.523335,5.0,2.0,1.0,...,0.0,1.0,0.0,18.0,1.0,1.0,1.0,2.0,1.0,IRN
563995,2.0,3.760697,2.0,3.260359,-0.500338,5,829.330026,2.0,2.0,1.0,...,0.0,0.0,0.0,22.0,2.0,6.0,0.0,4.0,1.0,GEO
1467908,2.0,3.620819,2.0,4.175891,0.555072,10,6147.847597,2.0,2.0,1.0,...,1.0,1.0,0.0,35.0,2.0,2.0,1.0,2.0,2.0,MMR
1205253,5.0,6.468249,5.0,6.728331,0.260082,8,20797.522176,5.0,2.0,1.0,...,1.0,1.0,0.0,22.0,1.0,1.0,1.0,2.0,5.0,PRY


In [26]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [27]:
Df = Df.sample(frac=1, random_state=42)

## Hyperparameter Fine-tuning

### Not Including Well-being

In [36]:
corr, p_value = pearsonr(Df['TreatmentEffectFemMal'], Df['Cantril_ladder'])
print(f"Pearson correlation coefficient: {corr}")
print(f"P-value: {p_value}")

Pearson correlation coefficient: 0.04016075152095992
P-value: 0.0


In [37]:
Df.shape

(1911212, 69)

In [38]:
y = Df['TreatmentEffectFemMal']

In [43]:
X = Df.drop(columns=['Cantril_ladder', 'MaleReal_y', 'TreatedAsMale', 'FemaleReal_y', 'TreatedAsFemale', 'TreatmentEffectFemMal'])

In [44]:
X.shape

(1911212, 63)

In [48]:
param_space = {
    'n_estimators': Integer(100, 5000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(3, 16),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [49]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True)

In [50]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [51]:
rkfcv = RandomRunNFoldsKFold(n_splits=10, run_splits=3, random_state=42)

In [52]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=20,
    scoring='r2',
    cv=rkfcv,
    n_jobs = 1,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [53]:
bayes_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:00:59] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=2129, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  40.0s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=2129, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  40.2s
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=7, min_child_weight=0.47928274405969296, n_estimators=2129, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time=  45.3s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=2.236420282054271, learning_rate=0.058429282697611454, max_delta_

BayesSearchCV(cv=RandomRunNFoldsKFold(n_splits=10, random_state=42, run_splits=None),
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, impo...
                             'min_child_weight': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=5000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [54]:
CV_result = bayes_search.cv_results_

In [55]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_delta_step,param_max_depth,param_min_child_weight,param_n_estimators,...,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,rank_train_score
11,272.434157,13.228172,0.438609,0.040364,0.010508,0.1,0.042074,16,8.578901,3631,...,0.839270,0.839227,0.000791,1,0.975921,0.975965,0.975911,0.975932,0.000023,3
14,102.863353,1.399579,0.234457,0.019484,0.001,0.1,10.0,13,0.00171,780,...,0.823826,0.824048,0.001399,2,0.958224,0.958037,0.958332,0.958198,0.000122,4
12,51.789467,0.524209,0.177794,0.009518,0.081506,0.1,0.403298,10,0.009229,2529,...,0.822925,0.823609,0.001143,3,0.904706,0.905141,0.904212,0.904687,0.000380,6
18,344.035736,2.265672,0.462437,0.014565,0.077188,0.023378,0.077769,16,1.585781,3017,...,0.815623,0.816050,0.000961,4,0.934980,0.935021,0.934965,0.934989,0.000024,5
6,144.504420,1.198789,0.365629,0.010967,0.29398,0.035541,0.027296,14,0.234681,3014,...,0.788751,0.788838,0.000747,5,0.848131,0.847466,0.847605,0.847734,0.000286,8
10,328.208658,0.938656,0.547339,0.067133,0.02378,0.042022,0.423663,16,0.010582,1808,...,0.784486,0.784666,0.001711,6,0.976681,0.976617,0.976614,0.976637,0.000031,2
7,66.606918,0.605275,0.235928,0.020025,0.149146,0.069186,0.096798,14,0.019781,351,...,0.777682,0.777977,0.000215,7,0.897771,0.896016,0.896550,0.896779,0.000734,7
17,793.785919,12.807586,1.222872,0.148293,0.110354,0.004539,0.12293,16,10.0,4281,...,0.756888,0.756916,0.000779,8,0.841469,0.841330,0.841580,0.841460,0.000103,9
19,399.581850,2.411277,0.560556,0.033637,0.001,0.080838,10.0,16,0.053547,2698,...,0.754118,0.754570,0.001145,9,0.998277,0.998273,0.998259,0.998270,0.000008,1
0,41.513440,2.530302,0.349997,0.131831,0.043693,0.02854,5.388551,7,0.479283,2129,...,0.722661,0.722208,0.000608,10,0.748182,0.747871,0.748026,0.748026,0.000127,10


In [56]:
dict(bayes_search.best_params_)

{'gamma': 0.010508194584096577,
 'learning_rate': 0.09999999999999999,
 'max_delta_step': 0.04207435943382148,
 'max_depth': 16,
 'min_child_weight': 8.578901446656365,
 'n_estimators': 3631,
 'reg_alpha': 1.7608714690874852,
 'reg_lambda': 0.001437060023125674,
 'subsample': 0.7109485535582907}

In [57]:
dump(bayes_search, 'Results/BayesSearchTreatmentEffect20iter.joblib')

['Results/BayesSearchTreatmentEffect20iter.joblib']

### One-hot encoding country

In [115]:
one_hot_encoded = pd.get_dummies(Df['COUNTRY_ISO3'], prefix='Country')

In [116]:
one_hot_encoded.head()

,Country_AFG,Country_AGO,Country_ALB,Country_ARE,Country_ARG,Country_ARM,Country_AUS,Country_AUT,Country_AZE,Country_BDI,...,Country_VEN,Country_VNM,Country_XKX,Country_XNC,Country_XNK,Country_XSR,Country_YEM,Country_ZAF,Country_ZMB,Country_ZWE
1751065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1120113,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1295575,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2247597,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2257066,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [117]:
Df_countryOneHot = pd.concat([Df, one_hot_encoded], axis=1)

In [118]:
Df_countryOneHot.shape

(1911212, 233)

In [119]:
y = Df_countryOneHot['TreatmentEffectFemMal']

In [120]:
X = Df_countryOneHot.drop(columns=['Cantril_ladder', 'MaleReal_y', 'TreatedAsMale', 'FemaleReal_y', 'TreatedAsFemale',
                                   'TreatmentEffectFemMal', 'COUNTRY_ISO3', 'Gender_female'])

In [121]:
X.shape

(1911212, 225)

In [127]:
param_space = {
    'n_estimators': Integer(1000, 5000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(5, 32),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [128]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True)

In [5]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [130]:
rkfcv = RandomRunNFoldsKFold(n_splits=10, run_splits=3, random_state=42)

In [131]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=20,
    scoring='r2',
    cv=rkfcv,
    n_jobs = 1,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [132]:
bayes_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.6min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.6min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.6min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=2.2

BayesSearchCV(cv=RandomRunNFoldsKFold(n_splits=10, random_state=42, run_splits=None),
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, impo...
                             'min_child_weight': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'n_estimators': Integer(low=1000, high=5000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [133]:
CV_result = bayes_search.cv_results_

In [134]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_delta_step,param_max_depth,param_min_child_weight,param_n_estimators,...,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,rank_train_score
18,440.867412,1.592450,4.147353,0.019159,0.012309,0.0393,0.954107,29,10.0,5000,...,0.815081,0.815060,0.001134,1,0.957115,0.957270,0.957243,0.957209,0.000068,4
15,282.587914,1.383444,3.322528,0.094039,0.022924,0.052636,3.423771,30,10.0,5000,...,0.809514,0.809891,0.001400,2,0.931880,0.931861,0.931842,0.931861,0.000016,5
11,303.624532,2.154597,3.095417,0.011604,0.001,0.1,10.0,32,10.0,5000,...,0.782802,0.782662,0.000826,3,0.917936,0.917917,0.917858,0.917904,0.000033,6
10,331.695216,2.028119,2.418583,0.056154,0.027465,0.036017,0.402192,18,0.239429,4504,...,0.773897,0.774197,0.000615,4,0.963111,0.963335,0.963741,0.963396,0.000261,3
17,192.613148,1.639741,1.798955,0.016982,0.023501,0.1,3.89068,32,10.0,5000,...,0.756705,0.756487,0.000777,5,0.970234,0.969992,0.970740,0.970322,0.000312,2
12,754.876238,4.452999,2.847991,0.064087,0.001,0.047046,2.671216,32,0.018312,1106,...,0.751090,0.750652,0.001513,6,0.995846,0.995833,0.995768,0.995816,0.000034,1
0,212.709173,1.552552,2.445241,0.167272,0.043693,0.02854,5.388551,14,0.479283,2656,...,0.750268,0.749213,0.000869,7,0.909940,0.910115,0.910785,0.910280,0.000364,7
9,153.613841,0.544207,2.060387,0.051893,0.001034,0.043218,0.92285,12,0.206375,2099,...,0.731332,0.731477,0.001427,8,0.894359,0.896129,0.893147,0.894545,0.001225,8
7,92.965530,0.761433,1.876750,0.191211,0.149146,0.069186,0.096798,28,0.019781,1205,...,0.728972,0.727634,0.002279,9,0.851056,0.854675,0.852373,0.852701,0.001496,9
13,881.929162,2.162182,5.094840,0.055639,0.050926,0.003777,5.132686,32,10.0,4112,...,0.677726,0.677591,0.000133,10,0.801877,0.801886,0.802576,0.802113,0.000327,10


In [135]:
dict(bayes_search.best_params_)

{'gamma': 0.01230893738822448,
 'learning_rate': 0.03929984746983189,
 'max_delta_step': 0.954107364480465,
 'max_depth': 29,
 'min_child_weight': 10.0,
 'n_estimators': 5000,
 'reg_alpha': 2.8833022521389937,
 'reg_lambda': 0.0016481010416658545,
 'subsample': 0.6431757974530777}

In [136]:
dump(bayes_search, 'Results/BayesSearchTreatmentEffectCountryOneHot20iter.joblib')

['Results/BayesSearchTreatmentEffectCountryOneHot20iter.joblib']

In [138]:
Df_countryOneHot.to_parquet('Data/GallupWB_Ml64var1911k14waveCounOneHot_v1.parquet')

In [6]:
bayes_search = load('Results/BayesSearchTreatmentEffectCountryOneHot20iter.joblib')

In [8]:
CV_result = bayes_search.cv_results_

In [9]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                    440.867412
std_fit_time                                                        1.59245
mean_score_time                                                    4.147353
std_score_time                                                     0.019159
param_gamma                                                        0.012309
param_learning_rate                                                  0.0393
param_max_delta_step                                               0.954107
param_max_depth                                                          29
param_min_child_weight                                                 10.0
param_n_estimators                                                     5000
param_reg_alpha                                                    2.883302
param_reg_lambda                                                   0.001648
param_subsample                                                    0.643176
params      

### Country Number

In [99]:
Df_counnum = Df.copy()

In [100]:
Df_counnum.shape

(1911212, 69)

In [101]:
Df_counnum['COUNTRY_ISO3'] = Df_counnum['COUNTRY_ISO3'].cat.codes

In [102]:
y = Df_counnum['TreatmentEffectFemMal']

In [103]:
X = Df_counnum.drop(columns=['Cantril_ladder', 'MaleReal_y', 'TreatedAsMale', 'FemaleReal_y', 'TreatedAsFemale', 'TreatmentEffectFemMal', 
                            'Gender_female'])

In [104]:
X.shape

(1911212, 62)

In [105]:
param_space = {
    'n_estimators': Integer(1000, 5000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(8, 32),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [106]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42)

In [107]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [108]:
rkfcv = RandomRunNFoldsKFold(n_splits=10, run_splits=3, random_state=42)

In [109]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=20,
    scoring='r2',
    cv=rkfcv,
    n_jobs = 1,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [110]:
bayes_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=16, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.7min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=16, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.7min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=16, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.7min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=2.2

BayesSearchCV(cv=RandomRunNFoldsKFold(n_splits=10, random_state=42, run_splits=None),
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, imp...
                             'min_child_weight': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'n_estimators': Integer(low=1000, high=5000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [111]:
CV_result = bayes_search.cv_results_

In [112]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_delta_step,param_max_depth,param_min_child_weight,param_n_estimators,...,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,rank_train_score
18,71.460504,0.518013,0.309706,0.003388,0.001,0.040999,0.151008,11,0.004462,1639,...,0.716989,0.717165,0.001275,1,0.865470,0.867340,0.866453,0.866421,0.000764,10
9,283.734763,1.903394,0.493512,0.035479,0.001034,0.043218,0.92285,14,0.206375,2099,...,0.715402,0.715534,0.000897,2,0.990978,0.990549,0.990804,0.990777,0.000176,6
10,818.421790,4.937563,1.142404,0.123187,0.001,0.045846,4.06536,32,10.0,4572,...,0.673241,0.672534,0.000501,3,0.995414,0.995426,0.995421,0.995420,0.000005,3
0,220.363311,0.610226,0.347949,0.009661,0.043693,0.02854,5.388551,16,0.479283,2656,...,0.661286,0.661123,0.000230,4,0.947406,0.947464,0.947699,0.947523,0.000127,7
19,1075.429962,5.362754,1.222405,0.068173,0.001,0.033887,5.713465,26,6.530476,4658,...,0.660204,0.660004,0.000155,5,0.996777,0.996774,0.996776,0.996776,0.000001,2
16,140.947351,1.057736,0.253451,0.004677,0.14532,0.030711,10.0,30,8.259051,4212,...,0.648461,0.647672,0.000636,6,0.800571,0.800594,0.802285,0.801150,0.000802,11
14,351.244639,3.227559,0.506408,0.029212,0.005257,0.047407,4.776889,18,2.105849,3596,...,0.647626,0.647337,0.000389,7,0.992135,0.992177,0.992207,0.992173,0.000029,5
17,91.567963,0.755376,0.177603,0.010519,0.115858,0.081756,10.0,29,0.101573,2989,...,0.638800,0.638654,0.001450,8,0.877283,0.878236,0.877859,0.877793,0.000392,9
7,107.305234,0.117828,0.172902,0.002972,0.149146,0.069186,0.096798,28,0.019781,1205,...,0.622856,0.623835,0.001703,9,0.887971,0.889971,0.887043,0.888328,0.001222,8
6,109.108511,0.057511,0.254918,0.003993,0.29398,0.035541,0.027296,29,0.234681,3378,...,0.605029,0.605882,0.001438,10,0.696559,0.695415,0.695044,0.695673,0.000645,12


In [113]:
dict(bayes_search.best_params_)

{'gamma': 0.001,
 'learning_rate': 0.040998678289102725,
 'max_delta_step': 0.1510080338435641,
 'max_depth': 11,
 'min_child_weight': 0.004462344581574194,
 'n_estimators': 1639,
 'reg_alpha': 0.013741272824756031,
 'reg_lambda': 0.012110581093122788,
 'subsample': 0.7990689195221465}

In [114]:
dump(bayes_search, 'Results/BayesSearchTreatmentEffect20iterCounnum.joblib')

['Results/BayesSearchTreatmentEffect20iterCounnum.joblib']

## Hyperparameter Fine-tuning Abs Value TE

### One-hot encoding country ABS Value TE

In [28]:
one_hot_encoded = pd.get_dummies(Df['COUNTRY_ISO3'], prefix='Country')

In [29]:
one_hot_encoded.head()

,Country_AFG,Country_AGO,Country_ALB,Country_ARE,Country_ARG,Country_ARM,Country_AUS,Country_AUT,Country_AZE,Country_BDI,...,Country_VEN,Country_VNM,Country_XKX,Country_XNC,Country_XNK,Country_XSR,Country_YEM,Country_ZAF,Country_ZMB,Country_ZWE
1751065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1120113,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1295575,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2247597,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2257066,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
Df_countryOneHot = pd.concat([Df, one_hot_encoded], axis=1)

In [31]:
Df_countryOneHot.shape

(1911212, 233)

In [34]:
Df_countryOneHot['TreatmentEffectFemMal'][:20]

1751065    0.807255
1120113    0.294307
1295575    0.036688
2247597    0.016840
2257066   -0.415478
957111     0.375183
455958    -0.264789
510066     0.272375
2060167    0.611200
1730352   -0.165969
1187704    0.240445
2137387   -0.492333
1458020   -0.082519
417605    -0.214852
1580292    0.167689
1792473    0.317164
2425619    0.046392
1094392    0.132968
2383227    0.011457
1469213    0.028362
Name: TreatmentEffectFemMal, dtype: float64

In [32]:
y = Df_countryOneHot['TreatmentEffectFemMal'].abs()

In [33]:
y[:20]

1751065    0.807255
1120113    0.294307
1295575    0.036688
2247597    0.016840
2257066    0.415478
957111     0.375183
455958     0.264789
510066     0.272375
2060167    0.611200
1730352    0.165969
1187704    0.240445
2137387    0.492333
1458020    0.082519
417605     0.214852
1580292    0.167689
1792473    0.317164
2425619    0.046392
1094392    0.132968
2383227    0.011457
1469213    0.028362
Name: TreatmentEffectFemMal, dtype: float64

In [35]:
X = Df_countryOneHot.drop(columns=['Cantril_ladder', 'MaleReal_y', 'TreatedAsMale', 'FemaleReal_y', 'TreatedAsFemale',
                                   'TreatmentEffectFemMal', 'COUNTRY_ISO3', 'Gender_female'])

In [36]:
X.shape

(1911212, 225)

In [37]:
param_space = {
    'n_estimators': Integer(1000, 5000),
    'learning_rate': Real(0.001, 0.1, prior='log-uniform'),
    'max_depth': Integer(5, 32),
    'subsample': Real(0.5, 1.0),
    'min_child_weight': Real(0.001, 10, prior='log-uniform'),
    'max_delta_step': Real(0.001, 10, prior='log-uniform'),
    'reg_lambda': Real(0.001, 10, prior='log-uniform'),
    'reg_alpha': Real(0.001, 10, prior='log-uniform'),
    'gamma': Real(0.001, 10, prior='log-uniform')
}

In [38]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True)

In [39]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [40]:
rkfcv = RandomRunNFoldsKFold(n_splits=10, run_splits=3, random_state=42)

In [41]:
bayes_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=param_space,
    n_iter=20,
    scoring='r2',
    cv=rkfcv,
    n_jobs = 1,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [42]:
bayes_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [16:16:21] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.3min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.3min
[CV] END gamma=0.04369339947510315, learning_rate=0.02853983686604182, max_delta_step=5.388550972627239, max_depth=14, min_child_weight=0.47928274405969296, n_estimators=2656, reg_alpha=0.025335258486348353, reg_lambda=0.9078559343576645, subsample=0.6522316555182531; total time= 3.2min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END gamma=2.236420282054271, learning_rate=0.058429282697611454, max_del

BayesSearchCV(cv=RandomRunNFoldsKFold(n_splits=10, random_state=42, run_splits=None),
              estimator=XGBRegressor(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, impo...
                             'min_child_weight': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'n_estimators': Integer(low=1000, high=5000, prior='uniform', transform='normalize'),
                             'reg_alpha': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'reg_lambda': Real(low=0.001, high=10, prior='log-uniform', transform='normalize'),
                             'subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')},
              verbose=2)

In [43]:
CV_result = bayes_search.cv_results_

In [44]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_learning_rate,param_max_delta_step,param_max_depth,param_min_child_weight,param_n_estimators,...,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,rank_train_score
17,662.069399,1.337428,3.574211,0.063210,0.001,0.052768,0.08578,31,2.36779,2824,...,0.620619,0.618436,0.002017,1,0.977516,0.977678,0.977516,0.977570,0.000076,2
16,392.605639,2.201235,2.443022,0.074700,0.00971,0.052771,0.199994,22,0.300512,3249,...,0.608905,0.606640,0.001950,2,0.964328,0.964347,0.963871,0.964182,0.000220,3
19,293.050868,0.482992,2.936027,0.010547,0.00164,0.076981,10.0,20,0.942929,5000,...,0.585342,0.582698,0.001929,3,0.822085,0.822499,0.822072,0.822219,0.000198,6
14,327.198087,0.282482,2.154274,0.077206,0.002678,0.1,0.129586,32,10.0,2245,...,0.583125,0.581533,0.001323,4,0.991102,0.991110,0.991136,0.991116,0.000015,1
0,194.661075,2.086649,2.342362,0.214530,0.043693,0.02854,5.388551,14,0.479283,2656,...,0.581789,0.580219,0.001133,5,0.830109,0.829278,0.828009,0.829132,0.000863,5
9,148.165673,1.024167,2.037487,0.025072,0.001034,0.043218,0.92285,12,0.206375,2099,...,0.568506,0.567181,0.001077,6,0.817000,0.818870,0.818872,0.818247,0.000882,7
10,221.893588,0.638368,1.817143,0.011803,0.017901,0.1,0.568172,32,4.406088,5000,...,0.561661,0.558024,0.002841,7,0.954282,0.954201,0.954106,0.954196,0.000072,4
7,73.610373,0.389744,1.541672,0.008423,0.149146,0.069186,0.096798,28,0.019781,1205,...,0.540660,0.539336,0.001699,8,0.718005,0.718815,0.718933,0.718584,0.000412,8
12,136.236710,0.504443,2.353360,0.017711,0.023782,0.02581,0.492083,12,0.003515,4660,...,0.524186,0.522584,0.001160,9,0.580396,0.579256,0.579143,0.579598,0.000566,10
13,46.690356,0.481806,1.517435,0.015175,0.0081,0.057093,10.0,9,0.419881,1222,...,0.501384,0.499610,0.001255,10,0.601729,0.602699,0.600774,0.601734,0.000786,9


In [45]:
dict(bayes_search.best_params_)

{'gamma': 0.001,
 'learning_rate': 0.05276773961416362,
 'max_delta_step': 0.0857797704073729,
 'max_depth': 31,
 'min_child_weight': 2.36778985445932,
 'n_estimators': 2824,
 'reg_alpha': 1.2645694578168394,
 'reg_lambda': 0.06727808895217463,
 'subsample': 0.9172598597711468}

In [46]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                    662.069399
std_fit_time                                                       1.337428
mean_score_time                                                    3.574211
std_score_time                                                      0.06321
param_gamma                                                           0.001
param_learning_rate                                                0.052768
param_max_delta_step                                                0.08578
param_max_depth                                                          31
param_min_child_weight                                              2.36779
param_n_estimators                                                     2824
param_reg_alpha                                                    1.264569
param_reg_lambda                                                   0.067278
param_subsample                                                     0.91726
params      

In [47]:
dump(bayes_search, 'Results/BayesSearchTreatmentEffectAbsCountryOneHot20iter.joblib')

['Results/BayesSearchTreatmentEffectAbsCountryOneHot20iter.joblib']